In [1]:
import requests  # 서버에 http 프로토콜로 요청
from bs4 import BeautifulSoup as bs  # html 파싱
import time  # 5초간 휴식 할 때 사용
from datetime import datetime  # 오늘 연월일 추출
import pandas as pd  # 데이터 프레임 만듬
from tqdm import tqdm
import sqlite3
from sqlalchemy import create_engine
import yes24_dbio as ydb

In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import time
from selenium.common.exceptions import NoSuchElementException

In [5]:
data = pd.read_csv('./data/yes24_08-23월간베스트.csv')

C:\Users\hi\AppData\Local\Temp\ipykernel_24372\1187745429.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./data/yes24_08-23월간베스트.csv')


In [6]:
colnames = ['yy', 'mon', 'image', 'category', 'title', 'title_sub', 'link', 'author', \
         'publisher', 'pub_date', 'price', 'n_reviews', 'detail_link', 'rating', 'tags']

In [7]:
data.columns = colnames

In [8]:
data

,yy,mon,image,category,title,title_sub,link,author,publisher,pub_date,price,n_reviews,detail_link,rating,tags
0,2008,1,https://image.yes24.com/goods/2599166/L,[도서],The Secret 시크릿,수 세기 동안 단 1%만이 알았던 부와 성공의 비밀,https://www.yes24.com/Product/Goods/2599166,"['론다 번', '김우열']",살림Biz,2007년 06월,"15,120",630,https://www.yes24.com/Product/Goods/2599166?Re...,7.5,"['#끌어당김', '#Y2K책장소환', '#크레마클럽에있어요', '#탑골공원책벌레'..."
1,2008,1,https://image.yes24.com/goods/2776013/L,[도서],마시멜로 두번째 이야기,NaN,https://www.yes24.com/Product/Goods/2776013,"['호아킴 데 포사다', '엘런 싱어', '공경희']",한국경제신문사(한경비피),2007년 12월,"10,800",202,https://www.yes24.com/Product/Goods/2776013?Re...,8.8,[]
2,2008,1,https://image.yes24.com/goods/2721480/L,[도서],리버보이,NaN,https://www.yes24.com/Product/Goods/2721480,"['팀 보울러', '정해영']",놀,2007년 10월,"8,100",234,https://www.yes24.com/Product/Goods/2721480?Re...,8.2,"['#국립어린이청소년도서관사서추천', '#올해의책']"
3,2008,1,https://image.yes24.com/goods/2783642/L,[도서],몰입 Think hard!,인생을 바꾸는 자기 혁명,https://www.yes24.com/Product/Goods/2783642,['황농문'],알에이치코리아(RHK),2007년 12월,"16,200",201,https://www.yes24.com/Product/Goods/2783642?Re...,8.0,"['#올해의책', '#생활습관']"
4,2008,1,https://image.yes24.com/goods/2752350/L,[도서],즐거운 나의 집,NaN,https://www.yes24.com/Product/Goods/2752350,['공지영'],푸른숲,2007년 11월,"8,820",328,https://www.yes24.com/Product/Goods/2752350?Re...,8.9,['#올해의책']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188959,2023,12,https://image.yes24.com/goods/117650984/L,[도서],초수익 성장주 투자,세계에서 가장 비밀스러운 주식 투자 시스템,https://www.yes24.com/Product/Goods/117650984,"['마크 미너비니', '김태훈', '김대현']",이레미디어,2023년 03월,"22,500",49,https://www.yes24.com/Product/Goods/117650984?...,9.6,['#크레마클럽에있어요']
188960,2023,12,https://image.yes24.com/goods/117585533/L,[도서],상처 주는 말 하는 친구에게 똑똑하게 말하는 법,"화내지 않고, 참지 않고, 울지 않는 마법의 표현 59",https://www.yes24.com/Product/Goods/117585533,"['김윤나', '유영근']",북라이프,2023년 02월,"11,700",52,https://www.yes24.com/Product/Goods/117585533?...,9.9,"['#학교생활이야기', '#슬기로운언어생활', '#친구관계', '#인간관계에대하여']"
188961,2023,12,https://image.yes24.com/goods/115033554/L,[도서],사람을 얻는 지혜,국내 최초 스페인어 완역본,https://www.yes24.com/Product/Goods/115033554,"['그라시안 이 모랄레스 발타사르', '김유경']",현대지성,2022년 10월,"10,800",103,https://www.yes24.com/Product/Goods/115033554?...,9.6,['#크레마클럽에있어요']
188962,2023,12,https://image.yes24.com/goods/114848694/L,[도서],인생의 역사,‘공무도하가’에서 ‘사랑의 발명’까지,https://www.yes24.com/Product/Goods/114848694,['신형철'],난다,2022년 10월,"16,200",111,https://www.yes24.com/Product/Goods/114848694?...,9.6,"['#2023서울국제도서전', '#YES24XEBS_대국민독서챌린지', '#2022..."


In [4]:
review_page = []

for links in data["link"]:

    # driver는 해상도에 맞춰서 빈 사이트를 띄워주는 로직 
    try:
        driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
        driver.set_window_size(1920, 1080) # 웹 브라우저 해상도 조절
        driver.get(links) # for 루프 안에 있는 link를 하나씩 넣으면서 페이지를 띄우게함 

        time.sleep(2)# 2초 쉬었다가 
        down_button = driver.find_element(By.CSS_SELECTOR,"#yDetailTabNavWrap > div > div.gd_tabBar > ul > li:nth-child(2) > a")
        down_button.click()
        # 댓글을 수집하기 위해서 배송/반품/교환 섹션으로 내려가게끔 페이지 다운 버튼 클릭 
        time.sleep(2)
        review_btn = driver.find_element(By.CSS_SELECTOR, '#infoset_reviewContentList > div:nth-child(3) > div.reviewInfoBot.crop > a > div')
        review_btn.click()
        time.sleep(3)
        # 도착한 포커스 내에서 페이지를 모두 저장 시킴(html_page라는 변수에)
        html_page = driver.page_source
        # 그 수집한 html_page 변수를 detail_page라는 리스트에 저장 시킴 
        detail_page.append(html_page)
        # 그 이후에 페이지 변수를 증가시킴 
        page = 1 
        nth_child = 4
        time.sleep(3)
        while True:
            print(f"페이지 : {page}, nth-child : {nth_child}, 페이지 개수 {len(detail_page)}")
            time.sleep(2)
            next_btn = driver.find_element(By.CSS_SELECTOR,f"#infoset_reviewContentList > div.review_sort.sortTop > div.review_sortLft > div > a:nth-child({nth_child})")
            next_btn.click()
            time.sleep(2)
            nth_child += 1
            page +=1 
            html_page = driver.page_source
            review_page.append(html_page)
            time.sleep(3)
            try: 
                empty_button = driver.find_element(By.CSS_SELECTOR,"#infoset_reviewContentList > div.review_sort.sortTop > div.review_sortLft > div > a.bgYUI.next")
                break
            except NoSuchElementException:
                pass

            if page % 5 == 0:
                time.sleep(2)
                next_prev_button = driver.find_element(By.CSS_SELECTOR,"#infoset_reviewContentList > div.review_sort.sortTop > div.review_sortLft > div > a.bgYUI.next")
                next_prev_button.click()
                page += 1 
                nth_child = 4
                html_page = driver.page_source
                review_page.append(html_page)
                time.sleep(3)
            
    except Exception :
        print(Exception)
        pass
    
    finally:
        driver.quit()
            
        # DB설계를 하고 해야함..
        # Book_id를 설정해야하는데 어케하는지모름

NameError: name 'data' is not defined